# Rigid Transformations

This *Jupyter Notebook* is dedicated to the study of **Rigid Transformations** and its properties.

The following study will be developed by the implementation of *Python* code for the application and visualization of the concepts learned with the use of *NumPy* and *Plotly* libraries. 

The following content is guided by the [Foundations of Robotics]((https://www.youtube.com/playlist?list=PLoWGuY2dW-Acmc8V5NYSAXBxADMm1rE4p)) course lectured by Oscar Ramos for the Universidad de Ingeniería y Tecnología (UTEC), 2018.

---

## Reference Frames

A Frame is essentially a way to stablish a spacial relation between elements. It can also be interpreted as the following:

- A **Coordinate System**;
- The base of a **Vector Space**;
- A representation of the **position** and **orientation** of a rigid body;

Important observations regarding frames:

- Completely defined by its **axes**;
- Its axes are usually unit vectors;
- Represented by the base of a vector space notation $\{F\}$;
- Elements represented in respect to a frame are superscripted by its name, e.g ${}^{F}E$.

---

### Rotation Matrices

For each frame"s perspective, their fundamental matrix that represents its axes will always be an identity matrix $I$ (or canonic). Therefore, to represent a frame in another frame"s perspective, it is essential to know the spatial relationship between these frames. 

This spatial relationship is described by the **Rotation Matrix** between these frames. For now, the frames will have the same origin points.

The rotation matrices will be represented as: $$\begin{bmatrix} | & | & | \\ \hat{x} & \hat{y} & \hat{z} \\ | & | & | \end{bmatrix}$$

Interpreting the world in a frame"s perspective is simple: assume that the known frame is represented by $I$. Seeing other frames in its perspective is just representing them by the rotation matrix that correlates the main frame in perspective to the observed frame. Mathematically:
1. Let a main frame $\{M\}$ be represented by $I$;
2. If an observed frame $\{O\}$ can be transformed into $\{M\}$ by a rotation matrix $R$, the frame $\{O\}$ will be represented by the rotation matrix ${}^{M}R_{O}$ in respect to the frame $\{M\}$;
3. Therefore, any given point ${}^{O}P$  can be viewed by the perspective of the frame $\{M\}$ by: $${}^{M}P = {}^{M}R_{O} \cdot {}^{O}P$$
4. The inverse action holds: $${}^{O}P = {}^{O}R_{M} \cdot {}^{M}P$$
5. Explicitly:
$${}^{M}R_{O}=\begin{bmatrix} \hat{x}_O \cdot \hat{x}_M & \hat{y}_O \cdot \hat{x}_M & \hat{z}_O \cdot \hat{x}_M  \\ \hat{x}_O \cdot \hat{y}_M & \hat{y}_O \cdot \hat{y}_M & \hat{z}_O \cdot \hat{y}_M  \\ \hat{x}_O \cdot \hat{z}_M & \hat{y}_O \cdot \hat{z}_M & \hat{z}_O \cdot \hat{z}_M \end{bmatrix}$$

---

### Properties of Rotation Matrices

Rotation matrices follow these properties:
- ${}^{B}R_{A}={}^{A}R_{B}^{-1}={}^{A}R_{B}^{T}$
- ${}^{A}R_{Z}={}^{A}R_{B}\cdot {}^{B}R_{C} \cdot ... \cdot {}^{X}R_{Y}\cdot{}^{Y}R_{Z}$
- $\det(R) = +1$

It is possible to interpret rotation matrices as rotation operators for elements. Therefore, applying the same rotation matrix to all points in a point cloud will result it to be rotated as such. In this case:
- Post-multiplication will be applied when rotating in the **current** frame;
- Pre-multiplication will be applied when rotating in the **fixed** frame;

---

### Homogeneous Transformations

To represent a point in a rotated *and* translated observed frame $O$, the following relationship is used: $${}^{O}P={}^{O}R_{M} \cdot {}^{M}P + {}^{O}t$$

In matrix representation: $$\begin{bmatrix} {}^{O}P \\ 1 \end{bmatrix} = \begin{bmatrix} {}^{O}R_{M} & {}^{O}t_{M} \\ 0 & 1 \end{bmatrix} \cdot \begin{bmatrix} {}^{M}P \\ 1 \end{bmatrix}$$ 

And in a compact way: $${}^{O}\tilde{P} = {}^{O}H_{M} \cdot {}^{M}\tilde{P}$$

Where ${}^{O}H$ is the homogeneous transformation to reference frame $O$ and the tilded coordinates are homogeneous coordinates.

A **Homogeneous Transformation Matrix** $H$ represents both position and orientation (pose) of a frame in relation to another in a single matrix, such that:

$$H = \begin{bmatrix} r_{11} & r_{12} & r_{13} & t_{x} \\ r_{21} & r_{22} & r_{23} & t_{y} \\ r_{31} & r_{32} & r_{33} & t_{z} \\ 0 & 0 & 0 & 1 \end{bmatrix} = \begin{bmatrix} R & t \\ 0 & 1 \end{bmatrix}$$

For the inverse of a homogeneous transformation, the following formula applies: $$H^{-1}= \begin{bmatrix} R^T & -R^T t \\ 0 & 1 \end{bmatrix}$$

The main purpose of using homogeneous transformations is to change the rotation and translation transformation from affine representation to linear representation, which is more convenient. 

---

In [2]:
# Importing modules...
import numpy as np
from scipy.spatial.transform import Rotation
from modules.viewer3d import *

In [3]:
# Make coordinates homogeneous
def to_homo(points):
    return np.vstack((points, np.ones(points.shape[1]))) 

# Joins the rotation and translation matrices to make a homogeneous transformation
def join_homo(R, t):
    return np.vstack((np.hstack((R, t)), np.array([0, 0, 0, 1])))

# Extract the rotation and translation matrices from the homogeneous transformation
def split_homo(H):
    return H[0:3, 0:3], H[0:3 , [-1]]

In [4]:
# Random rotation matrix
def random_rotation():
    rotation = np.eye(3)
    for axis in ["x", "y", "z"]:
        rotation = Rotation.from_euler(axis,  np.random.randint(0, 360), degrees=True).as_matrix() @ rotation

    return rotation

# Random translation matrix
def random_translation(L): # Confined in a cube with an edge of length L
    return np.array([[2 * L * np.random.random_sample() - L] for _ in range(3)])

In [5]:
# Returns the vertices of an unit cube with corner in origin
def cube(position=np.zeros((3, 1)), size=1):
    vertices = np.array([[(V>>2)&1, (V>>1)&1, (V>>0)&1] for V in range(8)]).T * size + position

    return vertices

In [6]:
# Create graph viewer
viewer = Viewer3D(
    title="Rotation of N points around an Axis", 
    size=2
)

# Plot main frame
viewer.add_frame(np.eye(4), "W")

# Point (1, 1, 1)
P = np.ones(3).reshape(-1, 1)

N = 6 # Number of points to visualize
axis = "z" # Axis of rotation

# Plot P rotated around the chosen axis N times
for theta in range(N):
    viewer.add_points(Rotation.from_euler(axis, 360*theta//N, degrees=True).as_matrix() @ P, f"{360*theta//N}°", "black");

viewer.figure.show(renderer="notebook_connected") 

In [7]:
# Create graph viewer
viewer = Viewer3D(
    title="Random Homogeneous Transformation", 
    size=5
)

# Plot main frame
viewer.add_frame(np.eye(4), "W")

box = cube() # Creates a cube
box = to_homo(box)

# Generate random homogeneous transformation
randH = join_homo(random_rotation(), random_translation(3)) 

# Rotate and translate in a homogeneous transformation
box = randH @ box 

box = box[:-1] # Extract regular coordinates

# Plot box
viewer.add_solid(box, name="Box", color="lightsteelblue")

# Generate and plot 3 random frames
for name in ['A', 'B', 'C']:
    viewer.add_frame(join_homo(random_rotation(), random_translation(3)), name)

viewer.figure.show(renderer="notebook_connected") 